# Build train and test matrices

The first thing to notice is that now, X and Y come from different dataframes. Since we want to use the GP image as a feature, but predict the value at each sensor. Maybe it can be used as a suplementary feature and combine both.

In order to avoid repeating the GP matrix when creating df_roll, we apply df_shift to df_idx, the dataframe connecting the datetime to the index of the GP array.

In [69]:
import pandas as pd
import numpy as np
import sys
import os
%load_ext autoreload
%autoreload 2

sys.path.append('../src/')
from utils.build_matrix import df_shift

PATH_DATA = '../../data'
if not os.path.isdir(PATH_DATA):
    print('The folder {} does not exist!'.format(PATH_DATA))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
try:
    df_gp = pd.read_pickle(os.path.join(PATH_DATA,'oahu_GP.pkl')) 
#     df.colums = data.colums.drop()
except:
    print('File {} does not exist.'.format(os.path.join(PATH_DATA,'oahu_GP.pkl')))

try:
    df_gp_torch = pd.read_pickle(os.path.join(PATH_DATA,'ghi_map_torch.pkl')) 
#     df.colums = data.colums.drop()
except:
    print('File {} does not exist.'.format(os.path.join(PATH_DATA,'ghi_map_torch.pkl')))
    
try:
    df_sensors = pd.read_pickle(os.path.join(PATH_DATA,'oahu_min_final.pkl')) 
#     df.colums = data.colums.drop()
except:
    print('File {} does not exist.'.format(os.path.join(PATH_DATA,'oahu_min_final.pkl')))

assert (df_sensors.index == df_gp.index).all(), 'The index is not the same'
df_idx = pd.DataFrame(data = list(range(len(df_gp))),index=df_gp.index,columns=['idx'],dtype='int')

# We load the info of the sensors to extract the longitude information
try:
    info = pd.read_pickle(os.path.join(PATH_DATA,'info.pkl')) 
#     df.colums = data.colums.drop()
except:
    print('File {} does not exist.'.format(os.path.join(PATH_DATA,'info.pkl')))

# Sorted longitudes
lon = info['Longitude'].sort_values(ascending=False).drop('AP3')
lat = info['Latitude'].sort_values(ascending=False).drop('AP3')
# Sort by lon
df_sensors[lon.index]


lon_list = df_gp.columns.levels[0].to_numpy()
lat_list = df_gp.columns.levels[1].to_numpy()
sensors_list = df_sensors.columns.to_numpy()
print('There are {} lons: {}'.format(len(lon_list),list(lon_list)))
print('There are {} lats: {}'.format(len(lat_list),list(lat_list)))
print('There are {} sensors: {}'.format(len(sensors_list),list(sensors_list)))

There are 12 lons: [-158.088, -158.087, -158.086, -158.085, -158.084, -158.083, -158.082, -158.081, -158.08, -158.079, -158.078, -158.077]
There are 9 lats: [21.308, 21.309, 21.31, 21.311, 21.312, 21.313, 21.314, 21.315, 21.316]
There are 16 sensors: ['AP1', 'AP4', 'AP5', 'AP6', 'AP7', 'DH1', 'DH10', 'DH11', 'DH2', 'DH3', 'DH4', 'DH5', 'DH6', 'DH7', 'DH8', 'DH9']


In [71]:
def df_shift(df, periods=1):
    return (pd.concat([df] + [ df.tshift(t+1, freq='1min') for t in range(periods) ], axis=1, 
                      keys=['t'] + [ 't-{:d}'.format(t+1) for t in range(periods) ],
                     names = ['time']+df.columns.names)
.dropna())

In [72]:
df_roll = df_shift(df_idx, periods=3)
df_roll.head()

time,t,t-1,t-2,t-3
,idx,idx,idx,idx
2010-03-19 14:18:00-10:00,3.0,2.0,1.0,0.0
2010-03-19 14:19:00-10:00,4.0,3.0,2.0,1.0
2010-03-19 14:20:00-10:00,5.0,4.0,3.0,2.0
2010-03-19 14:21:00-10:00,6.0,5.0,4.0,3.0
2010-03-19 14:22:00-10:00,7.0,6.0,5.0,4.0


In [73]:
def to_array(df_gp, df_sensors, periods):
    
    assert (df_sensors.index == df_gp.index).all(), 'The index is not the same'
    df_idx = pd.DataFrame(data = list(range(len(df_gp))),index=df_gp.index,columns=['idx'],dtype='int')
    
    df_gp.sort_index(axis=1)
    array_gp = df_gp.to_numpy().reshape(  [-1]+list(map(len,df_gp.columns.levels))   )

    array_sensors = df_sensors.to_numpy().reshape(  [-1]+[len(df_sensors.columns)]   )
    
    df_roll = df_shift(df_idx, periods=periods)
    
    df_train = df_roll[:'2011-07-31']
    df_test = df_roll['2011-08-01':]

    y_idx_train = df_train['t'].to_numpy(dtype='int').flatten()
    # Reverse the time index for the X, in case we use Recursive NN
    X_idx_train = df_train.drop(labels='t',axis=1,level='time').to_numpy(dtype='int')[:,::-1]

    y_idx_test = df_test['t'].to_numpy(dtype='int').flatten()
    # Reverse the time index for the X
    X_idx_test = df_test.drop(labels='t',axis=1,level='time').to_numpy(dtype='int')[:,::-1]
    
    
    X_train = array_gp[X_idx_train]
    y_train = array_sensors[y_idx_train]

    X_test = array_gp[X_idx_test]
    y_test = array_sensors[y_idx_test]
    
    return X_train, y_train, X_test, y_test

In [74]:
def to_array_persistance(df_gp, df_sensors, periods = 2):
    
    df_idx = pd.DataFrame(data = list(range(len(df_sensors))),index=df_sensors.index,columns=['idx'],dtype='int')


    array_sensors = df_sensors.to_numpy().reshape(  [-1]+[len(df_sensors.columns)]   )
    
    df_roll = df_shift(df_idx, periods=periods)
    
    df_train = df_roll[:'2011-07-31']
    df_test = df_roll['2011-08-01':]

    y_idx_train = df_train['t'].to_numpy(dtype='int').flatten()
    # Reverse the time index for the X, in case we use Recursive NN
    X_idx_train = df_train.drop(labels='t',axis=1,level='time').to_numpy(dtype='int')[:,::-1]

    y_idx_test = df_test['t'].to_numpy(dtype='int').flatten()
    # Reverse the time index for the X
    X_idx_test = df_test.drop(labels='t',axis=1,level='time').to_numpy(dtype='int')[:,::-1]
    
    
    X_train = array_sensors[X_idx_train]
    y_train = array_sensors[y_idx_train]

    X_test = array_sensors[X_idx_test]
    y_test = array_sensors[y_idx_test]
    
    return X_train, y_train, X_test, y_test

In [10]:
X_train, y_train, X_test, y_test = to_array(df_gp, df_sensors, periods=10)
print("Input shape: {}".format(X_train.shape[1:]))
print("Output shape: {}".format(y_train.shape[1:]))

Input shape: (10, 12, 9)
Output shape: (16,)


In [76]:
X_train, y_train, X_test, y_test = to_array_persistance(df_gp, df_sensors, periods=1)
print("Input shape: {}".format(X_train.shape[1:]))
print("Output shape: {}".format(y_train.shape[1:]))

Input shape: (1, 16)
Output shape: (16,)


In [80]:
y_te1 = y_test
pred = X_test.squeeze()
model_name = 'Persistence'
mae_val_list = np.mean(np.abs(pred-y_te1),axis=0)
test_loss = np.mean(mae_val_list)
# print('MAE_val ', cv_loss)
print('MAE_test ', test_loss)
print('MAE_test dissagregated', mae_val_list)

df = pd.DataFrame(data = mae_val_list, index=sensors_list, columns=[model_name])
path_results = os.path.join('../results',model_name+'.pkl')
df.to_pickle(path_results)


MAE_test  0.0962685147995401
MAE_test dissagregated [0.0961775  0.09573054 0.09783228 0.10107863 0.09506329 0.09337706
 0.09526633 0.09506355 0.09317234 0.09996936 0.09463232 0.09575586
 0.09724994 0.09551072 0.09539457 0.09902193]


## Define models

In [11]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape, Add, Multiply, Subtract, Dropout
from keras.layers import Conv2D, MaxPooling2D, LocallyConnected1D, Conv1D, UpSampling1D, MaxPooling1D, Dot, Concatenate
from keras.layers import TimeDistributed, LocallyConnected2D
from keras.layers import AveragePooling2D, LSTM
from keras import backend as K

Using TensorFlow backend.


Model architecture is defined below.

Some highlights:
* Locally connected works better than pure convolutional at the first layers (probably because the sensors at not located in a uniform grid)
* Trick to improve acc: add a final layer combining the convolutional prediction with the persistance prediction, so in case the input is "strange", the model could learn to output the persistance prediction (i.e., the previous time-step), which is somewhat reasonable

In [58]:
def make_model_LocCon2D_Dense( input_shape, n_sensors = 16):
    ''' 'GPconstant_LocCon2D_Dense'
    Input shape : (items, time_slice, longitude, latitude)
    '''
    xin = Input(shape=input_shape, name='GP_input')
    x = LocallyConnected2D(n_sensors, 3, data_format = 'channels_first', padding='valid',activation='relu', name='LocCon1')(xin)
    x = AveragePooling2D(data_format='channels_first')(x)
    x = LocallyConnected2D(n_sensors, 3, data_format = 'channels_first', padding='valid',activation='relu', name='LocCon2')(x)
    xo = TimeDistributed(Dense(2*n_sensors, name='TDDense',activation='relu'))(x)
    xo = Flatten(data_format='channels_first')(xo)
    xo = Dense(2*n_sensors, name='Dense1',activation='relu')(xo)
    xo = Dense(n_sensors, name='Dense2',activation='relu')(xo)
  
    x_final = Activation('relu', name='FinalRelu')(xo)
    
    model = Model(inputs=[xin], outputs=[x_final])
    return model

In [67]:
def make_model_rnn( input_shape, n_sensors = 16):
    ''' 'GPconstant_LocCon2D_LSTM'
    Input shape : (items, time_slice, longitude, latitude)
    '''
    xin = Input(shape=(None,)+input_shape[1:], name='GP_input')
    
    x_add_dim = Lambda(K.expand_dims)(xin)
    x = TimeDistributed( LocallyConnected2D(n_sensors,3, data_format = 'channels_last', padding = 'valid', activation ='relu')  )(x_add_dim)
    x = TimeDistributed( LocallyConnected2D(n_sensors//2,3, data_format = 'channels_last', padding = 'valid', activation ='relu')  )(x)
    x = TimeDistributed( Flatten() )(x)
    x = TimeDistributed( Dense(2*n_sensors) )(x)
    
    x1 = LSTM(5*n_sensors)(x)
    
#     xo = Flatten(data_format='channels_first')(x1)
    xo = Dense(2*n_sensors, name='Dense1',activation='relu')(x1)
    xo = Dense(n_sensors, name='Dense2',activation='relu')(xo)
  
    x_final = Activation('relu', name='FinalRelu')(xo)
    
    model = Model(inputs=[xin], outputs=[x_final])
    return model

In [68]:
model = make_model_rnn(X_tr1.shape[1:], n_sensors=y_train.shape[1])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
GP_input (InputLayer)        (None, None, 12, 9)       0         
_________________________________________________________________
lambda_9 (Lambda)            (None, None, 12, 9, 1)    0         
_________________________________________________________________
time_distributed_38 (TimeDis (None, None, 10, 7, 16)   11200     
_________________________________________________________________
time_distributed_39 (TimeDis (None, None, 8, 5, 8)     46400     
_________________________________________________________________
time_distributed_40 (TimeDis (None, None, 320)         0         
_________________________________________________________________
time_distributed_41 (TimeDis (None, None, 32)          10272     
_________________________________________________________________
lstm_9 (LSTM)                (None, 80)                36160     
__________

## Training

In [20]:
from sklearn.model_selection import TimeSeriesSplit
lr = 0.0001
opt = keras.optimizers.Adam(lr=lr)

# We add a callback to log metrics and another one to schedule the learning rate

#see clr.py in this same folder
from utils.clr import CyclicLR
c2 = CyclicLR(step_size=250, base_lr=lr)

batch_size = 2048   # as big as possible so we can explore many models
epochs = 40

In [45]:
import glob
from keras.models import load_model
import tensorflow as tf
def load_best_model(model_name, remove_rest = True):
    # Delete all the files but the final one
    list_filepath =  glob.glob(os.path.join("../models", model_name + "*"))
    f = lambda x: int(x.split('ep')[-1].split('_')[0])
    best_filepath = max(list_filepath,key= f)
    if remove_rest:
        for file in list_filepath:
            if file != best_filepath:
                print(file)
                os.remove(file)
    # Load the best model       
    model = load_model(best_filepath,custom_objects={'tf': tf})
    return model

In [62]:
def train_val_and_tess(df_gp, df_sensors, make_model_base, model_name='model'):
    X_tr1, y_tr1, X_te1, y_te1 = to_array(df_gp, df_sensors, periods=3)
    input_shape = X_tr1.shape[1:]
    n_sensors = y_tr1.shape[1]
    # Validation using TS split (just to obtain different MAE estimations, no hyperoptimization for the moment)
    cv_loss = []
    cv = TimeSeriesSplit(n_splits=5)
    for i, (tr_idx, va_idx) in enumerate(cv.split(X_tr1)):
        model = make_model_base(input_shape, n_sensors=n_sensors)
        model.compile(opt,
                      loss='mean_absolute_error')
        history_cv = model.fit(X_tr1[tr_idx], y_tr1[tr_idx],
                               batch_size=batch_size,
                               epochs=epochs,
                               validation_data=(np.atleast_3d(
                                   X_tr1[va_idx]), y_tr1[va_idx]),
                               callbacks=[c2],
                               shuffle=False,
                               verbose=0)
        cv_loss.append(history_cv.history['val_loss'][-1])
        print('Fold {}/{} ended!'.format(
            i+1, cv.get_n_splits()
        ))
    model_name = model_name + "_periods"+str(periods)
    filepath = os.path.join("../models", model_name +
                            "_ep{epoch:02d}_mae{val_loss:.3f}.h5")
    c1 = keras.callbacks.ModelCheckpoint(
        filepath,
        monitor='val_loss', verbose=1,
        save_best_only=True, save_weights_only=False,
        mode='auto', period=1)
    model = make_model(input_shape, n_sensors=n_sensors)
    model.compile(opt,loss='mean_absolute_error')
    history_test = model.fit(X_tr1, y_tr1,
                             batch_size=batch_size,
                             epochs=epochs,
                             validation_data=(X_te1, y_te1),
                             callbacks=[c1, c2],
                             shuffle=False,
                             verbose=0)

    model = load_best_model(model_name, remove_rest=True)
    pred = model.predict(X_te1)
    mae_val_list = np.mean(np.abs(pred-y_te1),axis=0)
    test_loss = np.mean(mae_val_list)
    print('MAE_val ', cv_loss)
    print('MAE_test ', test_loss)
    print('MAE_test dissagregated', mae_val_list)
    
    df = pd.DataFrame(data = mae_val_list, index=sensors_list, columns=[model_name])
    path_results = os.path.join('../results',model_name+'.pkl')
    df.to_pickle(path_results)
#     path_results = os.path.join('../results',model_name+'.csv')
#     df.to_csv(path_results)

    return test_loss, cv_loss, mae_val_list, model

In [63]:
def train_and_test(df_gp, df_sensors,to_array,make_model, periods, model_name ):
    X_tr1, y_tr1, X_te1, y_te1 = to_array(df_gp, df_sensors, periods=periods)
    input_shape = X_tr1.shape[1:]
    n_sensors = y_tr1.shape[1]
    model_name = model_name + "_periods"+str(periods)
    filepath = os.path.join("../models", model_name +
                            "_ep{epoch:02d}_mae{val_loss:.3f}.h5")
    c1 = keras.callbacks.ModelCheckpoint(
        filepath,
        monitor='val_loss', verbose=1,
        save_best_only=True, save_weights_only=False,
        mode='auto', period=1)
    model = make_model(input_shape, n_sensors=n_sensors)
    model.compile(opt,loss='mean_absolute_error')
    history_test = model.fit(X_tr1, y_tr1,
                             batch_size=batch_size,
                             epochs=epochs,
                             validation_data=(X_te1, y_te1),
                             callbacks=[c1, c2],
                             shuffle=False,
                             verbose=0)

    model = load_best_model(model_name, remove_rest=True)
    pred = model.predict(X_te1)
    mae_val_list = np.mean(np.abs(pred-y_te1),axis=0)
    test_loss = np.mean(mae_val_list)
    
    print('MAE_test ', test_loss)
    print('MAE_test dissagregated', mae_val_list)
    
    df = pd.DataFrame(data = mae_val_list, index=sensors_list, columns=[model_name])
    path_results = os.path.join('../results',model_name+'.pkl')
    df.to_pickle(path_results)
#     path_results = os.path.join('../results',model_name+'.csv')
#     df.to_csv(path_results)
    
    return test_loss, mae_val_list, model

## Testing models

In [54]:
make_model = make_model_rnn
periods = 3
model_name = 'GPtorch_LocCon2D_LSTM'
test_loss, mae_val_list, model = train_and_test(df_gp_torch, df_sensors,to_array,make_model, periods,model_name)


Epoch 00001: val_loss improved from inf to 0.10992, saving model to ../models/GPtorch_LocCon2D_LSTM_periods3_ep01_mae0.110.h5

Epoch 00002: val_loss improved from 0.10992 to 0.09950, saving model to ../models/GPtorch_LocCon2D_LSTM_periods3_ep02_mae0.100.h5

Epoch 00003: val_loss did not improve from 0.09950

Epoch 00004: val_loss improved from 0.09950 to 0.09338, saving model to ../models/GPtorch_LocCon2D_LSTM_periods3_ep04_mae0.093.h5

Epoch 00005: val_loss did not improve from 0.09338

Epoch 00006: val_loss did not improve from 0.09338

Epoch 00007: val_loss improved from 0.09338 to 0.09108, saving model to ../models/GPtorch_LocCon2D_LSTM_periods3_ep07_mae0.091.h5

Epoch 00008: val_loss improved from 0.09108 to 0.08986, saving model to ../models/GPtorch_LocCon2D_LSTM_periods3_ep08_mae0.090.h5

Epoch 00009: val_loss did not improve from 0.08986

Epoch 00010: val_loss did not improve from 0.08986

Epoch 00011: val_loss improved from 0.08986 to 0.08770, saving model to ../models/GPtorc

In [57]:
make_model = make_model_rnn
periods = 3
model_name = 'GPconstant_LocCon2D_LSTM'
test_loss, mae_val_list, model = train_and_test(df_gp, df_sensors,to_array,make_model, periods,model_name)


Epoch 00001: val_loss improved from inf to 0.15575, saving model to ../models/GPconstant_LocCon2D_LSTM_periods3_ep01_mae0.156.h5

Epoch 00002: val_loss did not improve from 0.15575

Epoch 00003: val_loss improved from 0.15575 to 0.09511, saving model to ../models/GPconstant_LocCon2D_LSTM_periods3_ep03_mae0.095.h5

Epoch 00004: val_loss improved from 0.09511 to 0.08862, saving model to ../models/GPconstant_LocCon2D_LSTM_periods3_ep04_mae0.089.h5

Epoch 00005: val_loss did not improve from 0.08862

Epoch 00006: val_loss did not improve from 0.08862

Epoch 00007: val_loss improved from 0.08862 to 0.08679, saving model to ../models/GPconstant_LocCon2D_LSTM_periods3_ep07_mae0.087.h5

Epoch 00008: val_loss improved from 0.08679 to 0.08591, saving model to ../models/GPconstant_LocCon2D_LSTM_periods3_ep08_mae0.086.h5

Epoch 00009: val_loss did not improve from 0.08591

Epoch 00010: val_loss improved from 0.08591 to 0.08562, saving model to ../models/GPconstant_LocCon2D_LSTM_periods3_ep10_mae0

In [55]:
make_model = make_model_rnn
periods = 5
model_name = 'GPtorch_LocCon2D_LSTM'
test_loss, mae_val_list, model = train_and_test(df_gp_torch, df_sensors,to_array,make_model, periods,model_name)


Epoch 00001: val_loss improved from inf to 0.10425, saving model to ../models/GPtorch_LocCon2D_LSTM_periods5_ep01_mae0.104.h5

Epoch 00002: val_loss improved from 0.10425 to 0.10044, saving model to ../models/GPtorch_LocCon2D_LSTM_periods5_ep02_mae0.100.h5

Epoch 00003: val_loss did not improve from 0.10044

Epoch 00004: val_loss improved from 0.10044 to 0.09864, saving model to ../models/GPtorch_LocCon2D_LSTM_periods5_ep04_mae0.099.h5

Epoch 00005: val_loss improved from 0.09864 to 0.09709, saving model to ../models/GPtorch_LocCon2D_LSTM_periods5_ep05_mae0.097.h5

Epoch 00006: val_loss did not improve from 0.09709

Epoch 00007: val_loss did not improve from 0.09709

Epoch 00008: val_loss improved from 0.09709 to 0.09348, saving model to ../models/GPtorch_LocCon2D_LSTM_periods5_ep08_mae0.093.h5

Epoch 00009: val_loss improved from 0.09348 to 0.09315, saving model to ../models/GPtorch_LocCon2D_LSTM_periods5_ep09_mae0.093.h5

Epoch 00010: val_loss did not improve from 0.09315

Epoch 000

In [56]:
make_model = make_model_rnn
periods = 5
model_name = 'GPconstant_LocCon2D_LSTM'
test_loss, mae_val_list, model = train_and_test(df_gp, df_sensors,to_array,make_model, periods,model_name)


Epoch 00001: val_loss improved from inf to 0.12824, saving model to ../models/GPconstant_LocCon2D_LSTM_periods5_ep01_mae0.128.h5

Epoch 00002: val_loss improved from 0.12824 to 0.09810, saving model to ../models/GPconstant_LocCon2D_LSTM_periods5_ep02_mae0.098.h5

Epoch 00003: val_loss improved from 0.09810 to 0.09093, saving model to ../models/GPconstant_LocCon2D_LSTM_periods5_ep03_mae0.091.h5

Epoch 00004: val_loss did not improve from 0.09093

Epoch 00005: val_loss did not improve from 0.09093

Epoch 00006: val_loss improved from 0.09093 to 0.08491, saving model to ../models/GPconstant_LocCon2D_LSTM_periods5_ep06_mae0.085.h5

Epoch 00007: val_loss improved from 0.08491 to 0.08405, saving model to ../models/GPconstant_LocCon2D_LSTM_periods5_ep07_mae0.084.h5

Epoch 00008: val_loss did not improve from 0.08405

Epoch 00009: val_loss improved from 0.08405 to 0.08371, saving model to ../models/GPconstant_LocCon2D_LSTM_periods5_ep09_mae0.084.h5

Epoch 00010: val_loss improved from 0.08371

In [48]:
make_model = make_model_rnn
periods = 9
model_name = 'GPtorch_LocCon2D_LSTM'
test_loss, mae_val_list, model = train_and_test(df_gp_torch, df_sensors,to_array,make_model, periods,model_name)


Epoch 00001: val_loss improved from inf to 0.11045, saving model to ../models/GPtorch_LocCon2D_LSTM_periods9_ep01_mae0.110.h5

Epoch 00002: val_loss improved from 0.11045 to 0.09824, saving model to ../models/GPtorch_LocCon2D_LSTM_periods9_ep02_mae0.098.h5

Epoch 00003: val_loss improved from 0.09824 to 0.09640, saving model to ../models/GPtorch_LocCon2D_LSTM_periods9_ep03_mae0.096.h5

Epoch 00004: val_loss did not improve from 0.09640

Epoch 00005: val_loss did not improve from 0.09640

Epoch 00006: val_loss improved from 0.09640 to 0.09580, saving model to ../models/GPtorch_LocCon2D_LSTM_periods9_ep06_mae0.096.h5

Epoch 00007: val_loss improved from 0.09580 to 0.09554, saving model to ../models/GPtorch_LocCon2D_LSTM_periods9_ep07_mae0.096.h5

Epoch 00008: val_loss did not improve from 0.09554

Epoch 00009: val_loss improved from 0.09554 to 0.09489, saving model to ../models/GPtorch_LocCon2D_LSTM_periods9_ep09_mae0.095.h5

Epoch 00010: val_loss improved from 0.09489 to 0.09349, savin

In [59]:
make_model = make_model_LocCon2D_Dense
periods = 3
model_name = "GPconstant_LocCon2D_Dense"
test_loss, mae_val_list, model = train_and_test(df_gp, df_sensors,to_array,make_model, periods,model_name)


Epoch 00001: val_loss improved from inf to 0.12976, saving model to ../models/GPconstant_LocCon2D_Dense_periods3_ep01_mae0.130.h5

Epoch 00002: val_loss improved from 0.12976 to 0.12712, saving model to ../models/GPconstant_LocCon2D_Dense_periods3_ep02_mae0.127.h5

Epoch 00003: val_loss did not improve from 0.12712

Epoch 00004: val_loss improved from 0.12712 to 0.12461, saving model to ../models/GPconstant_LocCon2D_Dense_periods3_ep04_mae0.125.h5

Epoch 00005: val_loss improved from 0.12461 to 0.12166, saving model to ../models/GPconstant_LocCon2D_Dense_periods3_ep05_mae0.122.h5

Epoch 00006: val_loss improved from 0.12166 to 0.12152, saving model to ../models/GPconstant_LocCon2D_Dense_periods3_ep06_mae0.122.h5

Epoch 00007: val_loss did not improve from 0.12152

Epoch 00008: val_loss improved from 0.12152 to 0.12074, saving model to ../models/GPconstant_LocCon2D_Dense_periods3_ep08_mae0.121.h5

Epoch 00009: val_loss improved from 0.12074 to 0.12032, saving model to ../models/GPconst

In [64]:
make_model = make_model_LocCon2D_Dense
periods = 5
model_name = "GPconstant_LocCon2D_Dense"
test_loss, mae_val_list, model = train_and_test(df_gp, df_sensors,to_array,make_model, periods,model_name)


Epoch 00001: val_loss improved from inf to 0.10458, saving model to ../models/GPconstant_LocCon2D_Dense_periods5_ep01_mae0.105.h5

Epoch 00002: val_loss improved from 0.10458 to 0.10244, saving model to ../models/GPconstant_LocCon2D_Dense_periods5_ep02_mae0.102.h5

Epoch 00003: val_loss did not improve from 0.10244

Epoch 00004: val_loss improved from 0.10244 to 0.09978, saving model to ../models/GPconstant_LocCon2D_Dense_periods5_ep04_mae0.100.h5

Epoch 00005: val_loss improved from 0.09978 to 0.09934, saving model to ../models/GPconstant_LocCon2D_Dense_periods5_ep05_mae0.099.h5

Epoch 00006: val_loss did not improve from 0.09934

Epoch 00007: val_loss did not improve from 0.09934

Epoch 00008: val_loss improved from 0.09934 to 0.09827, saving model to ../models/GPconstant_LocCon2D_Dense_periods5_ep08_mae0.098.h5

Epoch 00009: val_loss did not improve from 0.09827

Epoch 00010: val_loss did not improve from 0.09827

Epoch 00011: val_loss did not improve from 0.09827

Epoch 00012: val

In [60]:
make_model = make_model_LocCon2D_Dense
periods = 3
model_name = "GPtorch_LocCon2D_Dense"
test_loss, mae_val_list, model = train_and_test(df_gp_torch, df_sensors,to_array,make_model, periods,model_name)


Epoch 00001: val_loss improved from inf to 0.10719, saving model to ../models/GPtorch_LocCon2D_Dense_periods3_ep01_mae0.107.h5

Epoch 00002: val_loss improved from 0.10719 to 0.09786, saving model to ../models/GPtorch_LocCon2D_Dense_periods3_ep02_mae0.098.h5

Epoch 00003: val_loss improved from 0.09786 to 0.09631, saving model to ../models/GPtorch_LocCon2D_Dense_periods3_ep03_mae0.096.h5

Epoch 00004: val_loss did not improve from 0.09631

Epoch 00005: val_loss did not improve from 0.09631

Epoch 00006: val_loss improved from 0.09631 to 0.09409, saving model to ../models/GPtorch_LocCon2D_Dense_periods3_ep06_mae0.094.h5

Epoch 00007: val_loss did not improve from 0.09409

Epoch 00008: val_loss did not improve from 0.09409

Epoch 00009: val_loss improved from 0.09409 to 0.09218, saving model to ../models/GPtorch_LocCon2D_Dense_periods3_ep09_mae0.092.h5

Epoch 00010: val_loss improved from 0.09218 to 0.09177, saving model to ../models/GPtorch_LocCon2D_Dense_periods3_ep10_mae0.092.h5

Epo

In [61]:
make_model = make_model_LocCon2D_Dense
periods = 5
model_name = "GPtorch_LocCon2D_Dense"
test_loss, mae_val_list, model = train_and_test(df_gp_torch, df_sensors,to_array,make_model, periods,model_name)


Epoch 00001: val_loss improved from inf to 0.15288, saving model to ../models/GPtorch_LocCon2D_Dense_periods5_ep01_mae0.153.h5

Epoch 00002: val_loss improved from 0.15288 to 0.13103, saving model to ../models/GPtorch_LocCon2D_Dense_periods5_ep02_mae0.131.h5

Epoch 00003: val_loss improved from 0.13103 to 0.09758, saving model to ../models/GPtorch_LocCon2D_Dense_periods5_ep03_mae0.098.h5

Epoch 00004: val_loss improved from 0.09758 to 0.09426, saving model to ../models/GPtorch_LocCon2D_Dense_periods5_ep04_mae0.094.h5

Epoch 00005: val_loss did not improve from 0.09426

Epoch 00006: val_loss did not improve from 0.09426

Epoch 00007: val_loss improved from 0.09426 to 0.09235, saving model to ../models/GPtorch_LocCon2D_Dense_periods5_ep07_mae0.092.h5

Epoch 00008: val_loss did not improve from 0.09235

Epoch 00009: val_loss did not improve from 0.09235

Epoch 00010: val_loss did not improve from 0.09235

Epoch 00011: val_loss improved from 0.09235 to 0.09090, saving model to ../models/G

## There a lot of zeroes in AP1

In [380]:
df_sensors[(df_sensors == 0).sum(axis=1) != 0]

Location,AP1,AP4,AP5,AP6,AP7,DH1,DH10,DH11,DH2,DH3,DH4,DH5,DH6,DH7,DH8,DH9
Datetime,,,,,,,,,,,,,,,,
2011-10-31 14:57:00-10:00,0.0,0.303491,0.199010,0.222799,0.730146,0.235378,0.206958,0.195953,0.222458,0.197924,0.207120,0.215793,0.213093,0.218725,0.197835,0.200003
2011-10-31 14:58:00-10:00,0.0,0.210922,0.195017,0.240965,0.730983,0.246627,0.191409,0.182255,0.241759,0.182671,0.224530,0.220245,0.189771,0.202259,0.181770,0.179073
2011-10-31 14:59:00-10:00,0.0,0.300267,0.199470,0.277964,0.731833,0.351331,0.192025,0.182467,0.484857,0.176684,0.184388,0.193702,0.190857,0.305415,0.178547,0.259879
2011-10-31 15:00:00-10:00,0.0,0.684157,0.202666,0.219297,0.732695,0.724320,0.186316,0.186065,0.572306,0.177926,0.185465,0.192396,0.193652,0.310382,0.182195,0.187766
2011-10-31 15:01:00-10:00,0.0,0.625959,0.199096,0.202010,0.733570,0.678754,0.180199,0.179512,0.193857,0.171924,0.177501,0.182649,0.183730,0.200460,0.176388,0.172972
2011-10-31 15:02:00-10:00,0.0,0.197745,0.184069,0.203540,0.734458,0.192645,0.174070,0.178034,0.177689,0.170059,0.171246,0.179412,0.173742,0.178450,0.167124,0.161774
2011-10-31 15:03:00-10:00,0.0,0.205087,0.189390,0.368534,0.735360,0.172765,0.314819,0.258466,0.177907,0.671207,0.319159,0.686979,0.181494,0.171523,0.236762,0.160934
2011-10-31 15:04:00-10:00,0.0,0.757395,0.211734,0.673544,0.736275,0.614117,0.177279,0.181024,0.558267,0.170480,0.174264,0.183318,0.247375,0.182682,0.180489,0.184005
2011-10-31 15:05:00-10:00,0.0,0.340428,0.245633,0.584497,0.737205,0.691210,0.179094,0.188061,0.435888,0.173817,0.174916,0.181619,0.175070,0.452647,0.175527,0.163418
